Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2018.2

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de carga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [2]:
using Pkg
Pkg.add("Distributed")
using Distributed

# Adds workers
rmprocs(workers())
addprocs(3)

# Sets workload from worker_id to workload
@everywhere function set_workload(worker_id, workload)
    workloads[worker_id] = workload
end

# Notifies all other workers of my new workload
@everywhere function notify_my_workload_to_all_workers()
    for w in workers()
        if w != myid()
            remotecall(set_workload, w, myid(), workloads[myid()])
        end
    end
end

# Sets leader to new_leader and clears election variables
@everywhere function set_leader(new_leader)
    global leader = new_leader
    global participating = false
    global current_election = -1
    global current_election_workload = 1
end

# Circularly notifies that the new_leader was elected and finishes the election
@everywhere function finish_election(new_leader)
    set_leader(new_leader)

    if myid() == new_leader
        remotecall(println, 1, "New leader elected! Worker $new_leader")
        return
    end

    remotecall(finish_election, next_worker, new_leader)
end

# Circularly notifies that the new_leader wants to be a leader
@everywhere function elect(new_leader, new_leader_workload)
    # All workers were notified and agreed with the election
    if new_leader == myid()
        remotecall(finish_election, next_worker, new_leader)
        return
    end

    # There's another election happening and this new election request isn't
    # better than the current election if so, we discard this request
    if participating && (new_leader_workload > current_election_workload || (new_leader_workload == current_election_workload && new_leader <= current_election))
        return
    end
    
    # Thinks a new election should be enrolled where he is the new_leader
    if workloads[myid()] < new_leader_workload || (workloads[myid()] == new_leader_workload && myid() > new_leader)
        new_leader, new_leader_workload = myid(), workloads[myid()]
    end

    # Marks that it's participating in the election and saves the current election parameters
    global participating = true
    global current_election = new_leader
    global current_election_workload = new_leader_workload

    # Notifies the next worker about the election
    remotecall(elect, next_worker, new_leader, new_leader_workload)
end

# Checks if the eligibility conditions were meet, if so: enrolls the election
@everywhere function try_election()
    if participating
        return
    end

    if workloads[leader] >= 0.8 && workloads[myid()] < 0.8
        # println("Stared election")
        global participating = true
        remotecall(elect, next_worker, myid(), workloads[myid()])
    end
end

# Simulates the worker work
@everywhere function work()
    while true
        workloads[myid()] = round(rand(), digits = 1) # updates my workload

        notify_my_workload_to_all_workers()

        sleep(2)

        remotecall(println, 1, "Workload from Worker $(myid()): $([workloads[w] for w in sorted_workers])")

        try_election()
    end
end

# Initializes worker variables, election variables and starts its work
@everywhere function initialize_worker()
    global workloads = Dict([w => 0.0 for w in workers()])

    global sorted_workers = sort(workers())
    global next_worker = sorted_workers[findfirst(x -> x == myid(), sorted_workers) % length(sorted_workers) + 1]

    global leader = sorted_workers[1]
    global participating = false
    global current_election = -1
    global current_election_workload = 1

    work()
end

# Initializes all workers and sends them to work
@sync for w in workers()
    @spawnat w initialize_worker()
end

 Resolving package versions...
  Updating `C:\Users\ngome\.julia\environments\v1.1\Project.toml`
 [no changes]
  Updating `C:\Users\ngome\.julia\environments\v1.1\Manifest.toml`
 [no changes]


┌ Warning: rmprocs: process 1 not removed
└ @ Distributed C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Distributed\src\cluster.jl:928


Workload from Worker 4: [0.7, 0.2, 0.6]
Workload from Worker 2: [0.7, 0.2, 0.6]
Workload from Worker 3: [0.7, 0.2, 0.6]
Workload from Worker 4: [0.1, 0.0, 0.9]
Workload from Worker 2: [0.1, 0.0, 0.3]
Workload from Worker 3: [0.3, 0.0, 0.3]
Workload from Worker 4: [0.3, 0.2, 0.3]
Workload from Worker 2: [0.3, 0.2, 0.7]
Workload from Worker 3: [0.8, 0.2, 0.7]
New leader elected! Worker 3
Workload from Worker 4: [0.8, 0.8, 0.7]
New leader elected! Worker 4
Workload from Worker 2: [0.8, 0.8, 0.2]
Workload from Worker 3: [0.2, 0.8, 0.2]
Workload from Worker 4: [0.2, 0.6, 0.2]
Workload from Worker 2: [0.2, 0.6, 0.6]
Workload from Worker 3: [1.0, 0.6, 0.6]
Workload from Worker 4: [1.0, 0.7, 0.6]
Workload from Worker 2: [1.0, 0.7, 0.5]
Workload from Worker 3: [1.0, 0.7, 0.5]
Workload from Worker 4: [1.0, 0.0, 0.5]
Workload from Worker 2: [1.0, 0.0, 0.6]
Workload from Worker 3: [0.9, 0.0, 0.6]
Workload from Worker 4: [0.9, 0.5, 0.6]
Workload from Worker 2: [0.9, 0.5, 0.4]
Workload from Worker 3

InterruptException: InterruptException:

Workload from Worker 4: [0.3, 0.5, 0.3]
Workload from Worker 2: [0.3, 0.5, 0.6]
Workload from Worker 3: [0.7, 0.5, 0.6]
Workload from Worker 4: [0.7, 0.4, 0.6]
Workload from Worker 2: [0.7, 0.4, 0.3]
Workload from Worker 3: [0.8, 0.4, 0.3]
Workload from Worker 4: [0.8, 0.3, 0.3]
Workload from Worker 2: [0.8, 0.3, 0.4]
Workload from Worker 3: [0.6, 0.3, 0.4]
Workload from Worker 4: [0.6, 0.8, 0.4]
New leader elected! Worker 4
Workload from Worker 2: [0.6, 0.8, 0.1]
Workload from Worker 3: [0.4, 0.8, 0.1]
Workload from Worker 4: [0.4, 0.4, 0.1]
Workload from Worker 2: [0.4, 0.4, 0.6]
Workload from Worker 3: [0.2, 0.4, 0.6]
Workload from Worker 4: [0.2, 0.1, 0.6]
Workload from Worker 2: [0.2, 0.1, 0.8]
New leader elected! Worker 3
Workload from Worker 3: [0.9, 0.1, 0.8]
Workload from Worker 4: [0.9, 0.6, 0.8]
Workload from Worker 2: [0.9, 0.6, 0.5]
Workload from Worker 3: [0.5, 0.6, 0.5]
Workload from Worker 4: [0.5, 0.1, 0.5]
Workload from Worker 2: [0.5, 0.1, 0.4]
Workload from Worker 3

Workload from Worker 2: [0.4, 0.0, 0.3]
Workload from Worker 3: [1.0, 0.0, 0.3]
New leader elected! Worker 3
Workload from Worker 4: [1.0, 0.4, 0.3]
Workload from Worker 2: [1.0, 0.4, 0.7]
Workload from Worker 3: [0.3, 0.4, 0.7]
Workload from Worker 4: [0.3, 0.1, 0.7]
Workload from Worker 2: [0.3, 0.1, 0.3]
Workload from Worker 3: [1.0, 0.1, 0.3]
Workload from Worker 4: [1.0, 0.3, 0.3]
Workload from Worker 2: [1.0, 0.3, 0.2]
Workload from Worker 3: [0.2, 0.3, 0.2]
Workload from Worker 4: [0.2, 0.0, 0.2]
Workload from Worker 2: [0.2, 0.0, 0.1]
Workload from Worker 3: [0.7, 0.0, 0.1]
Workload from Worker 4: [0.7, 0.4, 0.1]
Workload from Worker 2: [0.7, 0.4, 0.5]
Workload from Worker 3: [0.9, 0.4, 0.5]
Workload from Worker 4: [0.9, 0.5, 0.5]
Workload from Worker 2: [0.9, 0.5, 0.2]
Workload from Worker 3: [0.2, 0.5, 0.2]
Workload from Worker 4: [0.2, 0.4, 0.2]
Workload from Worker 2: [0.2, 0.4, 0.4]
Workload from Worker 3: [0.4, 0.4, 0.4]
Workload from Worker 4: [0.4, 0.4, 0.4]
Workload fr

Workload from Worker 2: [0.6, 0.7, 0.3]
Workload from Worker 3: [0.4, 0.7, 0.3]
Workload from Worker 4: [0.4, 0.7, 0.3]
Workload from Worker 2: [0.4, 0.7, 0.1]
Workload from Worker 3: [0.3, 0.7, 0.1]
Workload from Worker 4: [0.3, 0.0, 0.1]
Workload from Worker 2: [0.3, 0.0, 0.6]
Workload from Worker 3: [0.5, 0.0, 0.6]
Workload from Worker 4: [0.5, 0.5, 0.6]
Workload from Worker 2: [0.5, 0.5, 0.7]
Workload from Worker 3: [0.9, 0.5, 0.7]
New leader elected! Worker 3
Workload from Worker 4: [0.9, 1.0, 0.7]
New leader elected! Worker 4
Workload from Worker 2: [0.9, 1.0, 0.5]
Workload from Worker 3: [0.2, 1.0, 0.5]
Workload from Worker 4: [0.2, 1.0, 0.5]
Workload from Worker 2: [0.2, 1.0, 0.6]
Workload from Worker 3: [0.0, 1.0, 0.6]
Workload from Worker 4: [0.0, 0.9, 0.6]
Workload from Worker 2: [0.0, 0.9, 0.5]
Workload from Worker 3: [0.5, 0.9, 0.5]
Workload from Worker 4: [0.5, 0.4, 0.5]
Workload from Worker 2: [0.5, 0.4, 0.4]
Workload from Worker 3: [0.2, 0.4, 0.4]
Workload from Worker 4


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.x](https://julialang.org/downloads/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.